# Download and save raw bibliographic data

We want to keep the part of the code that downloads the raw data separate from the rest. The reason is that we won't run it that many times.
There are two good reasons for this:
1. This is the code that takes most time, mainly because we need to wait for the API to serve the data.
2. We don't want to exceed our [weekly quota](https://dev.elsevier.com/api_key_settings.html) by performing repeated queries.

## Compose the search query

We need to put together a query string.
Here we specify a search phrase and (optionally) date filters, sorting and the amount of results to return.
Here you will find [Elsevier's instructions on how to write a query](https://dev.elsevier.com/tecdoc_federated_search.html).
A query is the *question* you ask the database.
The following example retrieves the occurrences of `Q Fever` from 2006 onwards.

In [ ]:
query = ['PUBYEAR > 2005 AND TITLE-ABS-KEY(Q Fever)']
query.append("view=COMPLETE")
query.append("count=25")  # Our subscription allows up to 25 results at a time

The following initializes the client and the doc_search object, and then executes the search.
You should request a private API key and create your own [configuration file as described here](https://github.com/ElsevierDev/elsapy/blob/master/CONFIG.md).
You should keep the file `config.json` private (i.e. out of a shared Git repository).

## Execute the search and save the results to a local file

The only variable that needs to be modified here is `filename`.
Note that if the file already exists the search results will be overwritten.

In [ ]:
filename = "my_results"

The following code downloads the results and saves it in 2 different formats:
* `.json` (the original format, as received from the API)
* `.xlsx` (an Excel file, convenient for quick examination)

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

with open("config.json") as con_file:
    config = json.load(con_file)    
client = ElsClient(config['apikey'])
doc_srch = ElsSearch('&'.join(query), 'scopus')
doc_srch.execute(client, get_all=True)

file = filename + ".json"
with open("output/{}".format(file), 'w') as f:
    json.dump(doc_srch.results, f)
print('Done saving {0} results to file "{1}".'.format(len(doc_srch.results), file))

First we initialize the interactive plotting (will be needed later).
In order to keep the rest of the code cleaner I will move all the imports up here.

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import pandas.io.json
import seaborn; seaborn.set()

from helperfuncs import extract_auth, extract_keywords, count_items

In [ ]:
query = ['PUBYEAR < 2006 AND TITLE-ABS-KEY(Q Fever)']
query.append("view=COMPLETE")
query.append("count=25")  # Our subscription allows up to 25 results at a time
client = ElsClient(config['apikey'])
doc_srch = ElsSearch('&'.join(query), 'scopus')
doc_srch.execute(client, get_all=True)
filename = "before2006.json"
with open("output/{}".format(filename), 'w') as json_file:
    json.dump(doc_srch.results, json_file)
print('Done saving {0} results to file "{1}".'.format(len(doc_srch.results), filename))

query = ['PUBYEAR > 2005 AND TITLE-ABS-KEY(Q Fever)']
query.append("view=COMPLETE")
query.append("count=25")  # Our subscription allows up to 25 results at a time
client = ElsClient(config['apikey'])
doc_srch = ElsSearch('&'.join(query), 'scopus')
doc_srch.execute(client, get_all=True)
filename = "after2005.json"
with open("output/{}".format(filename), 'w') as json_file:
    json.dump(doc_srch.results, json_file)
print('Done saving {0} results to file "{1}".'.format(len(doc_srch.results), filename))

## Clean up the raw data and remove invalid records

This section will depend on the data you obtain.
Our data set here requires very little cleaning, but it is useful as an example.
First we retrieve the results from the file we just created and load them into a *DataFrame* structure:

In [ ]:
filename = "my_results.json"
with open("output/{}".format(filename), 'r') as results_file:
    results = json.load(results_file)
df_results = pandas.io.json.json_normalize(results)
print('Loaded {0} results from file "{1}".'.format(len(results), filename))

Some records are "conference reviews", not articles, and are therefore not relevant.

In [ ]:
conference_review = df_results[df_results['subtype'] == "cr"].index
df_results.drop(conference_review, inplace=True)
print("Removed {0} 'Conference Review' records.".format(len(conference_review)))

A couple of records have no author data or title. We need to remove those as well.

In [ ]:
no_author_data = df_results[df_results['author'].isnull()].index
df_results.drop(no_author_data, inplace=True)
print("Removed {0} records missing author data.".format(len(no_author_data)))

no_title = df_results[df_results['dc:title'].isnull()].index
df_results.drop(no_title, inplace=True)
print("Removed {0} records with no title:".format(len(no_title)))
number_of_papers = len(df_results.index)

print("Proceeding with the remaining ({0}) records".format(number_of_papers))

After removing the invalid records, it is convenient to create a new DataFrame containing only the fields (columns) we are interested in visualizing.

In [ ]:
rec_date = pd.to_datetime(df_results['prism:coverDate'])
rec_auth = df_results['author'].map(extract_auth)
rec_kw = df_results['authkeywords'].map(extract_keywords)
rec_title = df_results['dc:title']
dict_of_series = {"Author": rec_auth,
                  "Date": rec_date,
                  "Keywords": rec_kw,
                  "Num_of_keywords": rec_kw.map(count_items),
                  "Title": rec_title,
                  "Year": rec_date.map(lambda x: x.year)}
df = pd.DataFrame(dict_of_series)

## Generate the visualizations

All the following plots will use matplotlib.
Using Seaborn is optional, but the plots look (subjectively) better.

Figure 1 shows the amount of papers (and related keywors) per year.

In [ ]:
papers_per_year = df['Year'].value_counts().sort_index()
keywords_per_year = df.groupby("Year")["Num_of_keywords"].sum()
fig02 = plt.figure()
ax02 = fig02.add_subplot(1, 1, 1)
ax02.set_title("Figure 1: Total number of papers and keywords per year (1990-2016)")
ax02.plot(papers_per_year, 'b-')
ax02.set_ylabel('Total number of papers per year', color='b')
ax02.tick_params('y', colors='b')

ax02b = ax02.twinx()
ax02b.plot(keywords_per_year, 'r--')
ax02b.set_ylabel('Total number of keywords per year', color='r')
ax02b.tick_params('y', colors='r')
